## Downloading Hadoop

In [ ]:
%env HADOOP_VERSION hadoop-2.9.2

In [ ]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/${HADOOP_VERSION}/${HADOOP_VERSION}.tar.gz -q --show-progress

### Extracting compressed files and removing .tar

In [ ]:
!tar -xvf ${HADOOP_VERSION}.tar.gz >/dev/null 
!rm       ${HADOOP_VERSION}.tar.gz

### Discovering Java path

In [ ]:
!dirname $(dirname $(readlink -f $(which javac)))

### Setting Java path envvar

We also added it to user's .bashrc so it will be loaded as the nodes perform ssh connections.

In [ ]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

In [ ]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " > .bashrc

# Using Hadoop in Standalone Mode (local)

# Using Hadoop in Pseudo-Distributed Mode

### Starting sshd server

Check postBuild and sshd_config files for more details

In [ ]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding to know hosts by establishing a ssh connectcion (avoiding yes/no host confirmation)

In [ ]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Copying configurations files to Hadoop folder

In [ ]:
!cp resources/configs/hadoop/* ${HADOOP_VERSION}/etc/hadoop/

### Formating the filesystem

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs namenode -format -force -nonInteractive

### Adding ssh options: running in a diferent port

In [ ]:
%env HADOOP_SSH_OPTS -p 8822 

### Starting/stoping NameNode daemon and DataNode daemon

In [ ]:
!./${HADOOP_VERSION}/sbin/start-dfs.sh
#!./${HADOOP_VERSION}/sbin/stop-dfs.sh

### Creating folders in the distributed file system

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/matheus/
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/matheus/input/

### Copying a file to a folder in the distributed file system

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -put ./resources/examples/newyorknewyork.txt /user/matheus/input/

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -ls /user/matheus/input/

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -cat /user/matheus/input/newyorknewyork.txt

# Running YARN on a single node

In [ ]:
!./${HADOOP_VERSION}/sbin/start-yarn.sh

# Simple word count example

In [ ]:
!./${HADOOP_VERSION}/bin/hadoop jar  ./${HADOOP_VERSION}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.9.2.jar wordcount \
                                      /user/matheus/input /user/matheus/output

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -ls /user/matheus/output/

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs dfs -cat /user/matheus/output/part-r-00000

In [ ]:
# !./${TARGET_HADOOP_VERSION}/bin/hdfs dfs -rm -r  /user/matheus/output/